In [1]:
from google.colab import drive
import os

import numpy as np
import matplotlib.pyplot as plt
import random, glob, io


import torch
import torch.nn as nn
from torch.utils.data import DataLoader


In [2]:
#Mount Google Drive to G-colab

def mountDrive():

  drive.mount('/content/drive')
  %cd ./drive/MyDrive/IVP_project/code

  print(os.listdir())

mountDrive()

Mounted at /content/drive
/content/drive/MyDrive/IVP_project/code
['dataset.py', 'images', '__pycache__', '.ipynb_checkpoints', 'DnCNN_GPU_V1.ipynb', 'Weights_DnCNN_S_SGD.pickle', 'DnCNN_GPU_V2.ipynb', 'pretrained', 'Weights_DnCNN_S_Adam_Gray.pickle', 'Weights_DnCNN_S_Adam.pickle', 'Exsisting.ipynb', 'DnCNN_GPU.ipynb']


In [3]:
"""
This block is to setup a code that will take care of displayig the images
"""

def imshow(I, title="Image"):
    plt.title(title)
    plt.imshow(I, cmap='gray')

def showImages(*Images,titles=[], figsize=(15,15)):
    I_count = len(Images)

    plt.subplots(1,I_count, figsize=figsize)

    for pos, I in enumerate(Images):
        plt.subplot(1,I_count,pos+1)
        try:
            imshow(I,titles[pos])
        except:
            imshow(I)
    
    plt.show()

# Model

In [4]:
class DnCNN(nn.Module):
    def __init__(self, channels, num_of_layers=17):
        super(DnCNN, self).__init__()
        kernel_size = 3
        padding = 1
        features = 64
        layers = []
        layers.append(nn.Conv2d(in_channels=channels, out_channels=features, kernel_size=kernel_size, padding=padding, bias=False))
        layers.append(nn.ReLU(inplace=True))
        for _ in range(num_of_layers-2):
            layers.append(nn.Conv2d(in_channels=features, out_channels=features, kernel_size=kernel_size, padding=padding, bias=False))
            layers.append(nn.BatchNorm2d(features))
            layers.append(nn.ReLU(inplace=True))
        layers.append(nn.Conv2d(in_channels=features, out_channels=channels, kernel_size=kernel_size, padding=padding, bias=False))
        self.dncnn = nn.Sequential(*layers)
    def forward(self, x):
        out = self.dncnn(x)
        return out

In [5]:
def getTrainedModel(model_name, imgChannels, num_of_layers= 17):
  model = DnCNN(channels=imgChannels, num_of_layers=num_of_layers)
  model = nn.DataParallel(model)
  model.load_state_dict(torch.load(f'./pretrained/{model_name}.pth'))
  return model
# model.state_dict()

In [6]:
def PSNR(original, compressed):
    mse = np.mean((original - compressed)**2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = original.max()
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

In [7]:
def testModel(model_name):

  test_dataset_path = "./pretrained/test/"
  im_list = []

  if model_name == 'DnCNN_B':
    DnCNN = getTrainedModel(model_name, imgChannels=1, num_of_layers=20) #Should be strictly 1
  else:
    DnCNN = getTrainedModel(model_name, imgChannels=1, num_of_layers=17) #Should be strictly 1

  gauss = lambda loc = 0, sigma = 1, shape=(180,180): np.random.normal(loc, sigma, shape)

  for file in glob.glob(f'{test_dataset_path}*.png'):
    img = plt.imread(file)
    if model_name == 'DnCNN_B':
      sigma = np.random.randint(0,55)
    else:
      sigma = int(model_name[len(model_name)-2:])
    noise = gauss(0,sigma,img.shape)/255
    noisy = torch.Tensor(img+noise).unsqueeze(0).unsqueeze(0)
    im_list.append((noisy, img))


  for noisy, clean in im_list:
    out = DnCNN(noisy)
    out = out.squeeze().squeeze().cpu()
    out = out.detach().numpy()

    noisy = noisy.squeeze().squeeze()

    denoised = (noisy-out).detach().numpy()

    psnr  = PSNR(clean, denoised)

    plt.figure()
    # titles = ['Noisy Image','True Noise', 'Learned Noise', 'Clean Image', f'Denoised Image\nPSNR: {psnr}']
    # showImages(noisy, noisy-clean, out, clean, denoised, titles=titles, figsize=(20,20))
    titles = ['Noisy Input','Clean Image', f'Denoised Image\nPSNR: {psnr}']
    showImages(noisy, clean, denoised, titles=titles, figsize=(20,20))

# Sigma = 15


In [8]:
testModel('DnCNN_S_15')

Output hidden; open in https://colab.research.google.com to view.

# Sigma = 25

In [9]:
testModel('DnCNN_S_25')

Output hidden; open in https://colab.research.google.com to view.

# Sigma = 50

In [10]:
testModel('DnCNN_S_50')

Output hidden; open in https://colab.research.google.com to view.

# Blind

In [11]:
testModel('DnCNN_B')

Output hidden; open in https://colab.research.google.com to view.